Script: Preprocess Data 
Author: Gabriel Geiger
Date: 17-01-2024 
<br>

Description: <br>
@input: Raw json files representing databases containing judgements, defendants and lay judges. 
@output: A merged dataframe where all nested data is flattened. Each row is a defendant with their corresponding case data (outcome, lay judges etc.)

In [1]:
import pandas as pd 
import warnings
import numpy as np 
import os
from pathlib import Path

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Get the notebook's directory
notebook_path = os.path.abspath('')+'/..'
os.chdir(notebook_path)

In [2]:
print(os.getcwd())

/Users/justin-casimirbraun/GitHub/norwegian_criminal_justice


Load Defendants

In [3]:
def load_defendants(file:str) -> pd.DataFrame : 
    df = pd.read_json(file)
    return df

defendant_df = load_defendants(os.getcwd() + "/00_raw_data/hidden/lr_tiltalte.json")

# Extract ID for defendants 
defendant_df["_id"] = defendant_df["_id"].apply(lambda id_dict: id_dict["$oid"])

# Add defendant prefix to all columns to avoid column collision with lay judges 
defendant_df = defendant_df.add_prefix("defendant_")


print("Defendant dataframe shape",defendant_df.shape)

Defendant dataframe shape (8094, 24)


Load Cases

In [4]:
def load_cases(file:str) -> pd.DataFrame : 
    df = pd.read_json(file)

    return df

cases_df = load_cases(os.getcwd() + "/00_raw_data/hidden/lr_dommer.json")

# Extract id for cases 
cases_df["_id"] = cases_df["_id"].apply(
    lambda id : id["$oid"]
)

print("Case Dataframe shape",cases_df.shape)

Case Dataframe shape (9019, 8)


Case Data Validation 

Run data validation steps and throw out any rows that violate business rules. 

In [5]:
# All cases should have a judgement, drop any that do not. 
length_before = len(cases_df)
cases_df.dropna(subset=["dom"],inplace=True)
print("{n} cases have been dropped because they did not have a judgement.".format(n=length_before - len(cases_df)))

# All cases should have one judge, drop any that do not. 
length_before = len(cases_df)
cases_df.dropna(subset=["fagdommere"],inplace=True)
cases_df = cases_df[cases_df["fagdommere"].apply(
    lambda judge_array: len(judge_array) == 1)
]

# There appears to have been some parsing on the Norwegians-side. 
cases_df = cases_df[cases_df["fagdommere"].apply(
    lambda judge_array : type(judge_array) == list
)]
print("{n} cases have been dropped because they did not have one judge.".format(n=length_before - len(cases_df)))

# All cases should have two lay judges, drop any that do not. 
length_before = len(cases_df)
cases_df.dropna(subset=["meddommere"],inplace=True)
cases_df = cases_df[cases_df["meddommere"].apply(
    lambda lay_array: len(lay_array) == 2)
]
print("{n} cases have been dropped because they did not have two lay judges.".format(n=length_before - len(cases_df)))

# All cases should have at least one defendant
length_before = len(cases_df)
cases_df.dropna(subset=["tiltalte"],inplace=True)
cases_df = cases_df[cases_df["tiltalte"].apply(
    lambda lay_array: len(lay_array) > 0)
]
print("{n} cases have been dropped because they did not have a defendant.".format(n=length_before - len(cases_df)))

# Reset the index. 
cases_df.reset_index(inplace=True)

print("Shape",cases_df.shape)

78 cases have been dropped because they did not have a judgement.
127 cases have been dropped because they did not have one judge.
36 cases have been dropped because they did not have two lay judges.
11 cases have been dropped because they did not have a defendant.
Shape (8767, 9)


Match Cases with Courts 

In [8]:
court_df = pd.read_json("00_raw_data/hidden/lr_filnavn.json")
court_df["_id"] = court_df["_id"].apply(lambda id_dict : id_dict["$oid"])

court_df["court_code"] = court_df["filnavn"].apply( 
    lambda filename : filename.split("-")[2].upper()
)

court_names = pd.read_csv("02_config/shortnames_districtcourts.csv",delimiter=";")
court_names.rename(columns={"Short name":"court_code"},inplace=True)

court_df = pd.merge(court_df,court_names,on="court_code",how="left")
court_df["District Court"].fillna(court_df["court_code"],inplace=True)

court_df["District Court"] = court_df["District Court"].apply(
    lambda court_name : court_name if "tingrett" in court_name else "Unknown"
)
court_df.rename(columns={"District Court":"case_district_court"},inplace=True)

cases_df = pd.merge(cases_df,court_df,on="_id",how="left")


/var/folders/wg/vgss5c890n11fcs6qdgphwfw0000gn/T/ipykernel_12373/250446799.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  court_df["District Court"].fillna(court_df["court_code"],inplace=True)


Merge Case DataFrame with Defendants DataFrame 

Each defendant in the defendant database has an ID. Each case in the case dataframe has an array of defendant ids. Ultimately we want each row in our final dataframe to be a defendant and their corresponding case. 

In [9]:
"""
Takes the raw 
"""
def parse_defendant_ids(raw_defendant_data:dict) -> list : 
    defendant_ids = []

    for defendant in raw_defendant_data : 
        if "motpart_id" in defendant : 
            if "$oid" in defendant["motpart_id"] : 
                defendant_ids.append(defendant["motpart_id"]['$oid'])
        
    
    return defendant_ids
        
# Create a clean list of defendant ids associated with the case 
cases_df["defendant_ids"] = cases_df["tiltalte"].apply(parse_defendant_ids)

# Explode the 'defendant_ids' column in cases_df to create separate rows for each defendant
cases_exploded = cases_df.explode('defendant_ids')

# Merge the exploded cases_df with defendants_df based on defendant_id
merged_df = pd.merge(cases_exploded, defendant_df, left_on='defendant_ids', right_on='defendant__id', how='left',suffixes=('_case', '_defendant'))
merged_df.reset_index(inplace=True,drop=True)

# Drop all cases without a defendant. Unfortunately not every defendant has an ID 
length_before = len(merged_df)
merged_df.dropna(subset=["defendant__id"],inplace=True)
merged_df.reset_index(inplace=True,drop=True)
print("{i} rows have been dropped because no matching defendant was found.".format(i=length_before - len(merged_df)))

print("Shape",merged_df.shape)


1113 rows have been dropped because no matching defendant was found.
Shape (8175, 37)


Process Judgement Data 

In [10]:
# Flatten the dom column
judgement_df = pd.json_normalize(data=merged_df["dom"],meta=["dømt","frifunnet","uvisst"])

# Create a dummy variable for frifunnet (aquitted) 
judgement_df["judgement_aquitted"] = judgement_df["frifunnet"].apply(
    lambda a : False if isinstance(a,float) else True)

# Create a dummy variable for uvisst (unknown) 
judgement_df["judgement_unknown"] = judgement_df["uvisst"].apply(
    lambda u : False if isinstance(u,float) else True)

# Create a dummy variable for dømt (convicted)
judgement_df["judgement_convicted"] = judgement_df["dømt"].apply(
    lambda u : False if isinstance(u,float) else True)

# Merge the dataframes 
length_before_merge = len(merged_df)
merged_df = pd.merge(merged_df, judgement_df, left_index=True, right_index=True, how='inner')
merged_df.reset_index(inplace=True,drop=True)

# Make sure no merging errors 
assert length_before_merge == len(merged_df)

Profesional Judges 

The only variable here behind function (which is always just profesional judge) is whether the profesional judge is or isn't in the majority. Note, this variable is n/a if the decision is unanimous. 

In [11]:
def parse_judge_data(raw_judge_data:list) : 
    judge_dict = raw_judge_data[0]

    if "flertall_eller_mindretall" in judge_dict : 
        if judge_dict["flertall_eller_mindretall"] == "flertall" : 
            return "Majority"

        else : 
            return "Minority"
    
    else : 
        return None 

merged_df["judge_majority_or_minority"] = merged_df["fagdommere"].apply(parse_judge_data)

Indictment

The sections under the indictment are in nested json of an arbitrary depth, so it's a bit tricky to unpack it all. Ultimately, we want to turn all indictment sections into dummy variables. 

An important note: Not all the sections in the indictment nessicairly pertain to all defendants in the case. 

Profesional Judges

In [12]:
def load_profesional_judges(filename) : 
    return pd.read_json(filename)

def load_judge_to_case(filename) : 
    return pd.read_json(filename)

def parse_judge_id(id) : 
    if isinstance(id,float) : 
        return None 
    
    if len(id) == 0 :
        return None 
    
    if id == [{}] : 
        return None 
    
    if "$oid" not in id[0]['fagdommer_id'] : 
        return None 
    
    else : 
        return id[0]['fagdommer_id']["$oid"]


judge_case_df = load_judge_to_case(os.getcwd() + "/00_raw_data/hidden/lr_dommer_fagdommerid.json")
judge_df = load_profesional_judges(os.getcwd() + "/00_raw_data/hidden/lr_fagdommere.json")

judge_case_df["case_id"] = judge_case_df["_id"].apply(
    lambda id : id["$oid"]
)

judge_case_df["judge_id"] = judge_case_df["fagdommere"].apply(
    parse_judge_id
)

judge_df["judge_id"] = judge_df["_id"].apply(
    lambda id : id["$oid"]
)

# Merge our two dataframes
judge_df = pd.merge(judge_case_df,judge_df,on="judge_id",how="left")
judge_df.drop(columns=["_id_x","_id_y","fagdommere"],inplace=True)
judge_df = judge_df.add_prefix("professional_judge_")

# Merge with our main dataframe 
merged_df = pd.merge(merged_df,judge_df,left_on="_id",right_on="professional_judge_case_id",how="left")
merged_df.drop(columns=["professional_judge_case_id"],inplace=True)


In [13]:

"""
Recursively flatten the nested json 
"""
def flatten_judgement_sections(nested_list) -> list:
    if (isinstance(nested_list,float)) or (nested_list == None) : 
        return nested_list 

    # If the judgement is unknown it is labeled "Fant ikke ord"
    if isinstance(nested_list,str) : 
        return [nested_list]
    
    flattened_list = []
    
    for item in nested_list:
        if isinstance(item, list):
            flattened_list.extend(flatten_judgement_sections(item))
        else:
            flattened_list.append(item)

    return flattened_list

def dummify_sections(row_sections:list,section:str) -> bool : 
    if isinstance(row_sections,float) : 
        return False 
    
    elif section in row_sections : 
        return True 

    else : 
        return False 
    
# Flatten sections in tiltale (indictment)
merged_df["flattened_charges"] = merged_df["tiltale"].apply(flatten_judgement_sections)

# Explode it so we have all sections in the database in one place and take unique
section_counts = merged_df["flattened_charges"].explode().value_counts()

# Filter for all values above threshold (Note: Set to 0 for now because we want to include everything)
filtered_values = section_counts[section_counts > 0].index.tolist()

# Turn all sections with a count above the threshold into a dummy variable (For now all sections)
for section in filtered_values:
    merged_df["charged_" + section] = merged_df['flattened_charges'].apply(
         dummify_sections,args=(section,)
    ).astype(int)


### Extract Aggravating and Mitigating Circumstances. 

In [14]:

"""
Recursively flatten the nested json 
"""
def flatten_circumstances(nested_list) -> list:
    if (isinstance(nested_list,float)) or (nested_list == None) : 
        return nested_list 

    # If the judgement is unknown it is labeled "Fant ikke ord"
    if isinstance(nested_list,str) : 
        return [nested_list]
    
    flattened_list = []
    
    for item in nested_list:
        if isinstance(item, list):
            flattened_list.extend(flatten_circumstances(item))
        else:
            flattened_list.append(item)

    return flattened_list

def dummify_circumstances(row_circumstances:list,circumstance:str) -> bool : 
    if isinstance(row_circumstances,float) : 
        return False 
    
    elif circumstance in row_circumstances : 
        return True 

    else : 
        return False 


aggravating_mitigating = pd.read_json("00_raw_data/hidden/130524_judgements_mitigating_aggravating.json")

aggravating_mitigating['_id'] = aggravating_mitigating['_id'].apply(
    lambda id : id['$oid']
)

# Create boolean variable that captures whether a case has aggravating or mitigating circumstances 
aggravating_mitigating['has_aggravating'] = aggravating_mitigating['skjerpende'].apply(
    lambda agg : True if isinstance(agg,list) else False 
)

aggravating_mitigating['has_mitigating'] = aggravating_mitigating['formildende'].apply(
    lambda mit : True if isinstance(mit,list) else False 
)

# Flatten aggravating and mitigating circumstnaces 

# Flatten sections in tiltale (indictment)
aggravating_mitigating["flattened_aggravating"] = aggravating_mitigating["skjerpende"].apply(flatten_circumstances)

aggravating_mitigating['flattened_mitigating'] = aggravating_mitigating["formildende"].apply(flatten_circumstances)

aggravating_list = aggravating_mitigating["flattened_aggravating"].explode().unique().tolist()
mitigating_list = aggravating_mitigating["flattened_mitigating"].explode().unique().tolist()

for circumstance in aggravating_list:
    if isinstance(circumstance,float) : 
        continue 

    aggravating_mitigating["aggravating_" + circumstance] = aggravating_mitigating['flattened_aggravating'].apply(
         dummify_circumstances,args=(circumstance,)
    ).astype(int)

for circumstance in mitigating_list:
    if isinstance(circumstance,float) : 
        continue 

    aggravating_mitigating["mitigating_" + circumstance] = aggravating_mitigating['flattened_mitigating'].apply(
         dummify_circumstances,args=(circumstance,)
    ).astype(int)


merged_df = pd.merge(merged_df, aggravating_mitigating, on="_id")

Matching Defendant Data

The case data contains an array called 'defendants' with some information about each defendant in the case (punishment, whether they were aquitted, etc.). For our purposes, we only want each row to have the relevant case defendant data and punishment, not the data for all the other defendants in the case. 

In [15]:

"""
Matches the nested defendant data with the defendant matched to this row. 
"""
def match_defendant_data(row) -> dict : 
    defendant_list = row["tiltalte"]

    for defendant in defendant_list: 
        if "motpart_id" in defendant : 
            if defendant["motpart_id"]["$oid"] == row["defendant__id"] : 
                return defendant

"""
Function for flattening the nested json data related to the defendant. 
Creates two intermediate variables: convicted_raw and punishment_raw that are unpacked 
seperately 
"""
def parse_defendant_data(raw_defendent_data:dict) : 
    defendant_data = {"convicted_raw":None,
                      "defendant_aquited":None,
                      "punishment_raw":None}

    if "dømt" in raw_defendent_data : 
        defendant_data["convicted_raw"] = raw_defendent_data["dømt"]
    
    if "frifunnet" in raw_defendent_data : 
        defendant_data["defendant_aquited"] = True 
    
    if "straff" in raw_defendent_data : 
        defendant_data["punishment_raw"] = raw_defendent_data["straff"]

    return pd.Series(defendant_data)

"""
Function for unpacking them raw punishment data into a set of variables
"""
def parse_punishment_data(raw_punishment_data:dict) : 
    new_punishment_columns = {"betinget":None,
                              "fengsel_dager":None,
                              "fengsel_dager_betinget":None,
                              "fengsel_subsidiært_dager":None,
                              "bot":None,
                              "forvaring":None,
                              "amfunnsstraff_gjennomføringstid_dager":None,
                              "samfunnsstraff_timer":None}

    # Return if no punishment 
    if raw_punishment_data == None : 
        return pd.Series(new_punishment_columns)
    
    # Otherwise loop through our new columns and update them if there's a match
    for col in new_punishment_columns : 
        if col in raw_punishment_data : 
            new_punishment_columns[col] = raw_punishment_data[col]
    
    return pd.Series(new_punishment_columns)

# Find our matching defendant data and unpack it. 
merged_df["raw_matched_defendant_data"] = merged_df.apply(match_defendant_data,axis=1)
merged_df[["convicted_raw","defendant_aquitted","punishment_raw"]] = merged_df["raw_matched_defendant_data"].apply(
    parse_defendant_data
)

"""
We reuse the approach for indictments for flattening and dummifying conviction sections
"""
# Flatten sections in conviction 

merged_df.convicted_raw.fillna(value=np.nan, inplace=True)

merged_df["flattened_convictions"] = merged_df["convicted_raw"].apply(flatten_judgement_sections)

# Explode it so we have all sections in the database in one place and take unique
section_counts = merged_df["flattened_convictions"].explode().unique()

# Turn all sections with a count above the threshold into a dummy variable 
for section in section_counts:
    if isinstance(section,str) : 
        merged_df["convicted_" + section] = merged_df['flattened_convictions'].apply(
            dummify_sections,args=(section,)
        ).astype(int)

# Finally we unpack the punishment data 
merged_df[["betinget",
          "fengsel_dager",
          "fengsel_dager_betinget",
          "fengsel_subsidiært_dager",
          "bot",
          "forvaring",
          "amfunnsstraff_gjennomføringstid_dager",
          "samfunnsstraff_timer"]] = merged_df["punishment_raw"].apply(parse_punishment_data)



/var/folders/wg/vgss5c890n11fcs6qdgphwfw0000gn/T/ipykernel_12373/1715876295.py:68: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df.convicted_raw.fillna(value=np.nan, inplace=True)


Lay Judges

First we unpack the lay judge field in the case data 

In [16]:

"""
Parse the data for the lay judge field in the case data. 
"""
def parse_lay_judges(raw_lay_data:list) -> pd.Series : 

    parsed_lay_data = []
    
    # Loop through the lay judges
    for lay_dict in raw_lay_data : 

        # Append the id field 
        if "meddommer_id" in lay_dict : 
            if "$oid" in lay_dict["meddommer_id"] : 
                parsed_lay_data.append(lay_dict["meddommer_id"]["$oid"])
        
        else : 
            parsed_lay_data.append(None)

        # Whether the lay judge is in the majority or minority 
        if "flertall_eller_mindretall" in lay_dict : 
            if lay_dict["flertall_eller_mindretall"] == "flertall" : 
                parsed_lay_data.append("Majority")
            
            else : 
                parsed_lay_data.append("Minority")
        
        else : 
            parsed_lay_data.append(None)
    
    return pd.Series(parsed_lay_data)


merged_df[["lay_judge_1","lay_judge_1_majority_or_minority","lay_judge_2","lay_judge_2_majority_or_minority"]] = merged_df["meddommere"].apply(
    parse_lay_judges
)

Merge Lay Judges Dataframe 

Finally, we merge the lay judges database into a final, merged dataframe

In [17]:
def load_lay_judges(file:str) -> pd.DataFrame : 
    df = pd.read_json(file)

    return df

lay_df = load_lay_judges(os.getcwd() + "/00_raw_data/hidden/lr_meddommere.json")

# Extract ID for defendants 
lay_df["_id"] = lay_df["_id"].apply(lambda id_dict: id_dict["$oid"])

# Create copies for each lay df with the correct prefix 
lay_df_1 = lay_df.copy().add_prefix("lay_1_")
lay_df_2 = lay_df.copy().add_prefix("lay_2_")

# Merge based on lay judge 1 and lay judge 2 ids
merged_df = pd.merge(merged_df, lay_df_1, left_on='lay_judge_1', right_on='lay_1__id', how='left')
merged_df = pd.merge(merged_df, lay_df_2, left_on='lay_judge_2', right_on='lay_2__id', how='left')

Match sections to legal groupings 

In [19]:
def dummify_chapters(row_convictions:list,section:str) -> bool : 
    if isinstance(row_convictions,float) : 
        return False 
    
    elif section in row_convictions : 
        return True

    else : 
        return False     

penal_code = pd.read_excel("02_config/penal_code_key.xlsx")

# Create mapping between Chapters and Sections 
chapter_section_mapping = dict(zip(penal_code['Section Norwegian'], penal_code['Chapter']))

# Turn all Chapters into a dummy variable for convicted
for section,chapter in chapter_section_mapping.items():

    # Create dummy variables for Chapters related to charges 
    merged_df["chapter_charged_" + chapter.lower().replace(" ","_")] = merged_df['flattened_charges'].apply(
         dummify_chapters,args=(section,)
    ).astype(int)

    # Create dummy variables for Chapters related to conviction  
    merged_df["chapter_convicted_" + chapter.lower().replace(" ","_")] = merged_df['flattened_convictions'].apply(
         dummify_chapters,args=(section,)
    ).astype(int)

# Create mapping between English and Norwegian sections 
norwegian_english_mapping = dict(zip(penal_code['Section Norwegian'], penal_code['section_name']))

# Add Prefixes 
mapping_charged = {'charged_' + str(key): "charged_" + value.replace(" ","_").lower().replace(",","") \
 for key, value in norwegian_english_mapping.items()
 }

mapping_convicted = {
    'convicted_' + str(key): "convicted_" + value.replace(" ","_").lower().replace(",","") \
    for key, value in norwegian_english_mapping.items()
    }

mapping_merged = {**mapping_charged,**mapping_convicted}
merged_df.rename(columns=mapping_merged,inplace=True)

Remove Intermediate Columns, Rename, and then Export

In [22]:

# A key mapping Norwegian columns to English translations 
column_key = eval(open("02_config/en_no_column_key.txt","r").read())

columns_to_drop = ["fagdommere",
                   "meddommere",
                   "tiltale",
                   "dom",
                   "tiltalte",
                   "dømt",
                   "frifunnet",
                   "uvisst",
                   "raw_matched_defendant_data",
                   "convicted_raw",
                   "punishment_raw",
                   "index"]


merged_df.reset_index(inplace=True,drop=True)
merged_df.drop(columns=columns_to_drop,inplace=True)
merged_df.rename(columns=column_key,inplace=True)

print(merged_df.shape)

merged_df.to_excel("01_preprocessed_data/hidden/preprocessed_data.xlsx",index=False)

(8175, 3057)
